In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from tqdm import tqdm
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys
import os

In [ ]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)

In [ ]:
def generate_examples(filepath):
    """Yields examples."""
    # Yields (key, example) tuples from the dataset
    with open(filepath, encoding="utf-8") as f:
        data = json.load(f)
        for example in data["data"]:
            title = example.get("title", "").strip()
            for paragraph in example["paragraphs"]:
                context = paragraph["context"].strip()
                for qa in paragraph["qas"]:
                    question = qa["question"].strip()
                    id_ = str(qa["id"])

                    answer_starts = [answer["answer_start"] for answer in qa["answers"]]
                    answers = [answer["text"].strip() for answer in qa["answers"]]

                    yield id_, {
                        "title": title,
                        "context": context,
                        "question": question,
                        "id": id_,
                        "answers": {
                            "answer_start": answer_starts,
                            "text": answers,
                        },
                    }

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("./output-eval-strategy/checkpoint-20000/")
tokenizer = AutoTokenizer.from_pretrained("./output-eval-strategy/checkpoint-20000/")

In [ ]:
with open("./data/xquad/xquad.tr.json", "rb") as f:
    xquad = json.load(f) 

In [ ]:
nlp = pipeline(
    'question-answering', 
    model=model,
    tokenizer=tokenizer
)

In [ ]:
f1 = exact_match = total = 0
for article in tqdm(xquad["data"]):
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            total += 1

            ground_truths = list(map(lambda x: x['text'], qa['answers']))
            prediction = nlp({'question': qa["question"], 'context':  paragraph["context"]})["answer"]
    
            exact_match += metric_max_over_ground_truths(
                exact_match_score, prediction, ground_truths)
            f1 += metric_max_over_ground_truths(
                f1_score, prediction, ground_truths)

exact_match = 100.0 * exact_match / total
f1 = 100.0 * f1 / total

result = {'exact_match': exact_match, 'f1': f1}
print("Results on XQuAD", result)

In [ ]:
f1 = exact_match = total = 0

for subfolder in os.scandir("./data/thquad"):
    for data_file in os.scandir(subfolder):
        examples = generate_examples(data_file.path)
        for example in tqdm(examples):
            total += 1

            ground_truths = example[1]["answers"]["text"]
            prediction = nlp(
                {
                    'question': example[1]["question"],
                    'context':  example[1]["context"]
                }
            )
            prediction = prediction["answer"]
            exact_match += metric_max_over_ground_truths(
                exact_match_score, prediction, ground_truths)
            f1 += metric_max_over_ground_truths(
                f1_score, prediction, ground_truths)

exact_match = 100.0 * exact_match / total
f1 = 100.0 * f1 / total

result = {'exact_match': exact_match, 'f1': f1}
print("Results on THQuAD", result)